In [1]:
from tensorflow.keras.models import model_from_json
import numpy as np

import tensorflow as tf

class FacialExpressionModel(object):

    EMOTIONS_LIST = ["Angry", "Disgust",
                    "Fear", "Happy",
                    "Neutral", "Sad",
                    "Surprise"]

    def __init__(self, model_json_file, model_weights_file):
        # load model from JSON file
        with open(model_json_file, "r") as json_file:
            loaded_model_json = json_file.read()
            self.loaded_model = model_from_json(loaded_model_json)

        # load weights into the new model
        self.loaded_model.load_weights(model_weights_file)
        self.loaded_model.make_predict_function()

    def predict_emotion(self, img):
        self.preds = self.loaded_model.predict(img)
        return FacialExpressionModel.EMOTIONS_LIST[np.argmax(self.preds)]

In [24]:
import cv2
import numpy as np

facec = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
model = FacialExpressionModel("./model.json", "./model_weights.h5")
font = cv2.FONT_HERSHEY_SIMPLEX

class VideoCamera(object):
    def __init__(self, source = None):
           self.video = cv2.VideoCapture(source)

    def __del__(self):
        self.video.release()

    # returns camera frames along with bounding boxes and predictions
    def get_predict(self):
        _, fr = self.video.read()
        gray_fr = cv2.cvtColor(fr, cv2.COLOR_BGR2GRAY)
        faces = facec.detectMultiScale(gray_fr, 1.3, 5)

        for (x, y, w, h) in faces:
            fc = gray_fr[y:y+h, x:x+w]

            roi = cv2.resize(fc, (48, 48))
            pred = model.predict_emotion(roi[np.newaxis, :, :, np.newaxis])

            cv2.putText(fr, pred, (x, y), font, 1, (255, 255, 0), 2)
            cv2.rectangle(fr,(x,y),(x+w,y+h),(255,0,0),2)

        return fr
class Image(object):
    def __init__(self, source = None):
        self.path = source

    def __del__(self):
        self.image.release()

    # returns image along with bounding boxes and predictions
    def get_predict(self):
        img = cv2.imread(self.path)
        #img = cv2.resize(img,(0,0),fx = 0.3, fy = 0.3)
        # using when the image shape is too large
        gray_fr = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = facec.detectMultiScale(gray_fr, 1.3, 5)

        for (x, y, w, h) in faces:
            fc = gray_fr[y:y+h, x:x+w]

            roi = cv2.resize(fc, (48, 48))
            pred = model.predict_emotion(roi[np.newaxis, :, :, np.newaxis])

            cv2.putText(img, pred, (x, y), font, 1, (255, 255, 0), 2)
            cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)

        return img

In [25]:
import matplotlib.pyplot as plt
def predict_from_cam(camera):
    while True:
        frame = camera.get_predict()
        cv2.imshow('Facial Expression Recognization',frame)
        if cv2.waitKey(1) & 0xFF == ord('x'):
            break
    cv2.destroyAllWindows()
def predict_from_image(image):
        img_out = image.get_predict()
        cv2.imwrite('./img_pre.jpg',img_out)
        cv2.waitKey(1)
        cv2.destroyAllWindows()

In [ ]:
predict_from_cam(VideoCamera(source=0))

In [ ]:
predict_from_image(Image(source='./kanna_hashimoto_1.jpg'))